In [1]:
from utils import*

In [156]:
####################################################################################################################################################
class NODE(object):
    ################################################################################################################################################
    def __init__(self, markers_color, IDD=None, port=None):
        self.markers_color = markers_color
        self.IDD = IDD
        self.port = port

    ################################################################################################################################################
    def load_markers(self, dataset_name, file_name, window_length=11):   
      
        # Load raw data 
        markers_file_path = get_markers_file_path(dataset_name, file_name)  
        raw_df  = pd.read_csv(
            markers_file_path,                                                  # relative python path to subdirectory
            usecols = ['time', self.markers_color],                             # Only load the three columns specified.
            parse_dates = ['time'] ) 
        self.markers = pd.DataFrame()

        # DON'T USE!! markers are switched and smoothing causes error    
        # Processing
        # self.markers = self.markers.rolling(window_length, axis=0).mean()    # Gap filling
        # self.markers = self.markers.ffill(axis=0).bfill(axis=0)              # Smoothing

        # Time
        date_time = pd.to_datetime( raw_df['time'] , format=datime_format)
        self.markers['time'] = [ np.round( (datetime.combine(date.min, t.time())-datetime.min).total_seconds(), 2) for t in date_time]
        
        # Markers
        self.markers['markers'] = [list(map(float, l.replace(']','').replace('[','').replace('\n','').split(", "))) for l in raw_df[self.markers_color].values]        
#         self.markers['markers'] = [list(map(float, l.replace(']','').replace('[','').replace('\n','').replace(',','').split(" "))) for l in raw_df[self.markers_color].values]
#         self.markers['markers'] = raw_df[self.markers_color]
        return
    ################################################################################################################################################
    def load_rssi(self, dataset_name, file_name, window_length=11):
        # Load data 
        rssi_file_path = get_rssi_file_path(dataset_name, file_name)       
        raw_df = pd.read_csv(
            rssi_file_path,                                                     # relative python path to subdirectory
            delimiter  = ';',                                                   # Tab-separated value file.
            usecols = ['IDD', 'Time', 'Ant/RSSI'],                              # Only load the three columns specified.
            parse_dates = ['Time'] )                                            # Intepret the birth_date column as a date      

        # self.rssi.loc[ self.rssi.IDD != self.IDD,'rssi'] = np.nan
        raw_df = raw_df.loc[ raw_df['IDD'] == self.IDD, :]
        self.rssi = pd.DataFrame({ 'rssi': raw_df['Ant/RSSI'].str.replace('Ant.No 1 - RSSI: ', '').astype(int) })

        # Processing
        self.rssi = self.rssi.rolling(window_length, axis=0).median()   # Smoothing
        self.rssi = self.rssi.ffill(axis=0).bfill(axis=0)               # Gap Filling
         
        # Time
        date_time = pd.to_datetime( raw_df['Time'] , format=datime_format)
        self.rssi['time'] = [ np.round( (datetime.combine(date.min, t.time())-datetime.min).total_seconds(), 2) for t in date_time]
        return
    ################################################################################################################################################
    def load_vind(self, dataset_name, file_name, window_length=11):
        pass
    ################################################################################################################################################
    def shift_time(self, shift):
        self.markers.time -= shift
        if self.IDD is not None: self.rssi.time -= shift
        if self.port is not None: self.rssi.time -= shift
        return
    ################################################################################################################################################
    def center(self, window_length=7, polyorder=1):
        markers_npy = self.markers.drop(['time'], axis=1).to_numpy()
        markers_npy = markers_npy.reshape(np.shape(markers_npy)[0], -1, 3)
        center = np.mean(markers_npy, axis=1) 
        
        center = np.nan_to_num(center)
        center = signal.savgol_filter( center, window_length=window_length, polyorder=polyorder, axis=0)            
        return center
    ################################################################################################################################################    
    def norm(self, window_length=7, polyorder=1):
        markers_npy = self.markers.drop(['time'], axis=1).to_numpy()

        markers_npy =  markers_npy.reshape(np.shape(markers_npy)[0], -1, 3)
        v1 = markers_npy[:,1,:] - markers_npy[:,0,:]
        v2 = markers_npy[:,2,:] - markers_npy[:,0,:] 
        norm = np.cross( v1, v2)
        # norm[ norm[:,2]<0, :] *= -1 

        # norm = np.nan_to_num(norm)
        # norm = signal.savgol_filter( norm, window_length=window_length, polyorder=polyorder, axis=0)  
        norm = norm / ( np.reshape(np.linalg.norm(norm, axis=1), (-1,1)) * np.ones((1,3))) 
        return norm


In [163]:
dataset_name = 'dataset_04'
file_name = 'record_' + "{0:0=2d}".format(0)

In [172]:
node = NODE(markers_color='blue', IDD='E002240002749F45')
node.load_markers(dataset_name, file_name)
node.markers

,time,markers
0,42540.45,"[0.021102494, 0.06478552, 1.3510926, 0.1086278..."
1,42540.60,"[0.015856605, 0.059480865, 1.3404398000000002,..."
2,42540.67,"[0.10205695, 0.05062528, 1.3704771, 0.05843881..."
3,42540.74,"[0.01300617, 0.059854175999999995, 1.3410195, ..."
4,42540.81,"[0.011824125, 0.05870588, 1.3408794, 0.1003628..."
...,...,...
498,42579.73,"[0.026193475, -0.022328338, 1.1332632, -0.0281..."
499,42580.04,"[0.022273201, -0.027453125, 1.1225876000000001..."
500,42580.26,"[0.023233924, -0.03505238, 1.1375991, 0.061633..."
501,42580.36,"[0.023154112, -0.03798533, 1.1355872999999999,..."


In [ ]:
# dataset_name = 'dataset_00'

# time_folder_path = main_directory + '/dataset/' + dataset_name + '/kinect/time'
# markers_folder_path = main_directory + '/dataset/' + dataset_name + '/kinect/markers'

# for n in range(30):
#     file_name = 'record_' + "{0:0=2d}".format(n)
    
#     time_file_path = time_folder_path + '/' + file_name + '.txt'   
#     with open(time_file_path , 'r') as f:  lines = f.read().splitlines() 
#     date_time = pd.to_datetime( lines , format=datime_format)
#     time = pd.DataFrame({'time':[t.time() for t in date_time]})
             
#     locations_dict_ = dict()
#     for color in ['red','blue','green']:
#         markers_file_path = markers_folder_path + '/' + file_name + '_' + color +'.txt' 
#         locations = pd.read_csv( markers_file_path, delimiter="\t", header=None, dtype=np.float64).to_numpy()
# #         locations_dict_.update({color: locations})
#         locations_dict_.update({color: list(map(list,locations))})
    
#     locations_dict = defaultdict(list)  
#     for l in range(len(time)):
#         for color in ['red','blue','green']:
#             locations_dict[color].append( locations_dict_[color][l] )                            

#   # MArkers (time and locations)
#     markers = pd.concat([time, pd.DataFrame(locations_dict)], axis=1)

#     # Save 
#     markers_file_path = get_markers_file_path(dataset_name, file_name)
#     create_folder(markers_file_path)
#     markers.to_csv(markers_file_path, index=False)
        